In [25]:
#installing libs
%pip install praw pandas vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [1]:
#importing required libs for scraping
import praw
import pandas as pd
from praw.models import MoreComments
import time
from datetime import datetime, timezone

In [ ]:
#reddit creds
reddit = praw.Reddit(
    user_agent="",
    client_id="", 
    client_secret=""
)

In [3]:
#using a reddit post
#https://www.reddit.com/r/IndiaSpeaks/comments/1fr7tyb/scrap_dealer_gifts_multiple_iphones_to_son_for/ - apple test
url = "https://www.reddit.com/r/IndiaSpeaks/comments/1fqhbsz/youtuber_ranveer_allahbadias_youtube_channel_got/"
submission = reddit.submission(url=url)

In [4]:
#getting all comments from a post
submission.comments.replace_more(limit=None)
posts = []
for top_level_comment in submission.comments.list():
    if isinstance(top_level_comment, MoreComments):
        continue
    if top_level_comment.author and "bot" not in top_level_comment.author.name.lower() and not top_level_comment.stickied:
        posts.append(top_level_comment.body)
posts = pd.DataFrame(posts,columns=["body"])
indexNames = posts[(posts.body == '[removed]') | (posts.body == '[deleted]')].index
posts.drop(indexNames, inplace=True)

In [5]:
posts

,body
0,I heard both his passwords where 1234567@ 💀
1,Hogya recover
2,It was the hackers who showed him the next doo...
3,"What this basically tells is that:\n\n""Life is..."
4,This is the exact reason why India should deve...
...,...
67,YouTube has decades worth of data plus backed ...
68,Yes they can join. But Youtube algo recommends...
69,1. This is completely unrelated to Ranbir's ch...
70,Lmao yoh are placing to much importance on tec...


In [3]:
#extracting all comments from top10 posts of a subreddit
subreddit = reddit.subreddit('technews')

all_comments = pd.DataFrame()
    

In [28]:
def extract_comments(submission):
    posts = []
    submission.comments.replace_more(limit=None)
    count = 0
    for top_level_comment in submission.comments.list():
        if isinstance(top_level_comment, MoreComments):
            continue
        if (top_level_comment.author and "bot" not in top_level_comment.author.name.lower() 
            and not top_level_comment.stickied):
            posts.append(top_level_comment.body.encode('utf-8', 'ignore').decode('utf-8'))
    
    return posts


In [29]:
def process_posts():
    global all_comments
    attempt = 0
    for submission in subreddit.rising(limit=15):
        try:
            print(f"Processing post: {submission.title}")
            posts = extract_comments(submission)
            posts_df = pd.DataFrame(posts, columns=["body"])
            indexNames = posts_df[(posts_df.body == '[removed]') | (posts_df.body == '[deleted]')].index
            posts_df.drop(indexNames, inplace=True)
            posts_df['post_title'] = submission.title
            posts_df['post_time'] = datetime.fromtimestamp(submission.created_utc, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

            all_comments = pd.concat([all_comments, posts_df], ignore_index=True)
        
        except praw.exceptions.RedditAPIException as e:
            if "RATELIMIT" in str(e):
                attempt += 1
                sleep_time = 120 * attempt
                print(f"Rate limit hit. Sleeping for {sleep_time} seconds. Error: {e}")
                time.sleep(sleep_time) 
                continue
            else:
                print(f"Encountered an error: {e}")
                continue
    
        time.sleep(60)


In [30]:
process_posts()

Processing post: The Optimus robots at Tesla’s Cybercab event were humans in disguise
Processing post: New Gmail Security Alert For 2.5 Billion Users As AI Hack Confirmed
Processing post: Apple made a huge macOS privacy promise four years ago, but it's still unfulfilled
Processing post: SpaceX successfully catches returning Starship booster 
Processing post: This AI can think like an engineer—and it just designed a spaceship engine
Processing post: TikTok Lays Off Hundreds of Staff—to Replace Them With AI
Processing post: Silicon Valley is debating if AI weapons should be allowed to decide to kill
Processing post: Hacked Robot Vacuums Across the U.S. Started Yelling Slurs
Processing post: Steam now tells gamers up front that they're buying a license, not a game | The company appears to be getting ahead of a California law going into force next year.
Processing post: OpenAI says Chinese gang tried to phish its staff
Processing post: Marriott settles with FTC, to pay $52 million over dat

In [31]:
all_comments

,body,post_title,post_time
0,I was admittedly a bit uncomfortable when my A...,The Optimus robots at Tesla’s Cybercab event w...,2024-10-13 18:08:39
1,"The Mechanical Turk, classic.",The Optimus robots at Tesla’s Cybercab event w...,2024-10-13 18:08:39
2,"Teleoperated, so not quite Russian robots.",The Optimus robots at Tesla’s Cybercab event w...,2024-10-13 18:08:39
3,I could see a transition phase where teleworke...,The Optimus robots at Tesla’s Cybercab event w...,2024-10-13 18:08:39
4,These robotics aren’t even particularly ground...,The Optimus robots at Tesla’s Cybercab event w...,2024-10-13 18:08:39
...,...,...,...
1192,Ok Ivan,Hacktivists Claim Responsibility for Taking Do...,2024-10-10 15:02:00
1193,First link states the name of the org. Second ...,Hacktivists Claim Responsibility for Taking Do...,2024-10-10 15:02:00
1194,Jesus Christ some people just can’t admit when...,Hacktivists Claim Responsibility for Taking Do...,2024-10-10 15:02:00
1195,You totally missed the point. A misleading tit...,Hacktivists Claim Responsibility for Taking Do...,2024-10-10 15:02:00


In [33]:
all_comments.to_csv('red_data_2.csv', encoding='utf-8', index=False)

Labelling Data

In [34]:
import pandas as pd

# Load the dataset
file_path = 'red_data_2.csv'
data = pd.read_csv(file_path)

# Check the first few rows to understand the structure
# data.head()

In [35]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to classify sentiment using VADER
def classify_sentiment_vader(text):
    sentiment_score = analyzer.polarity_scores(str(text))
    # Classify as negative, neutral, or positive based on compound score
    if sentiment_score['compound'] >= 0.05:
        return 2  # Positive
    elif sentiment_score['compound'] <= -0.05:
        return 0  # Negative
    else:
        return 1  # Neutral

# Apply the VADER sentiment classification to the 'body' column
data['score'] = data['body'].apply(classify_sentiment_vader)

# Check the results
data[['body', 'score']].head()


,body,score
0,I was admittedly a bit uncomfortable when my A...,0
1,"The Mechanical Turk, classic.",1
2,"Teleoperated, so not quite Russian robots.",1
3,I could see a transition phase where teleworke...,1
4,These robotics aren’t even particularly ground...,1


In [36]:
data.to_csv('red_data_3.csv')